## 1. The ten best-selling video games
<p>In this project, we'll explore the top 400 best-selling video games created between 1977 and 2020. We'll compare a dataset on game sales with critic and user reviews to determine whether or not video games have improved as the gaming market has grown.</p>

<h3 id="game_sales"><code>game_sales</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>platform</code></td>
<td>varchar</td>
<td>Gaming platform</td>
</tr>
<tr>
<td style="text-align:left;"><code>publisher</code></td>
<td>varchar</td>
<td>Game publisher</td>
</tr>
<tr>
<td style="text-align:left;"><code>developer</code></td>
<td>varchar</td>
<td>Game developer</td>
</tr>
<tr>
<td style="text-align:left;"><code>games_sold</code></td>
<td>float</td>
<td>Number of copies sold (millions)</td>
</tr>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Release year</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>critic_score</code></td>
<td>float</td>
<td>Critic score according to Metacritic</td>
</tr>
<tr>
<td style="text-align:left;"><code>user_score</code></td>
<td>float</td>
<td>User score according to Metacritic</td>
</tr>
</tbody>
</table>


In [2]:
%%sql
postgresql:///games
SELECT *
    FROM game_sales
    ORDER BY games_sold DESC
    LIMIT 10;

10 rows affected.


game,platform,publisher,developer,games_sold,year
Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017
Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


## 2. Missing review scores
<p>Using data from the <code>reviews</code> table to gain more insight on which games that were not given any reviews. </p>


In [4]:
%%sql 
SELECT count(gs.game) as count
    FROM game_sales as gs
left join reviews as r
on r.game = gs.game
where r.critic_score is null and r.user_score is null;

 * postgresql:///games
1 rows affected.


count
31


## 3. Years that video game critics loved
<p>Best years for video games according to what the critics think. </p>

In [ ]:
%%sql
SELECT year, round(avg(critic_score), 2) as avg_critic_score
    FROM game_sales as gs
inner join reviews as r
on r.game = gs.game
    GROUP BY year
    ORDER BY avg_critic_score DESC
    limit 10  
   

## 4. Was 1982 really that great?
<p>Updating the query and find out whether 1982 really was such a great year for video games.</p>

In [ ]:
%%sql 
SELECT year, round(avg(critic_score), 2) as avg_critic_score, COUNT(DISTINCT r.game) AS num_games
    FROM game_sales as gs
inner join reviews as r
on r.game = gs.game
    GROUP BY year
    HAVING COUNT(DISTINCT r.game) > 4
    ORDER BY avg_critic_score DESC
    limit 10;  

## 5. Years that dropped off the critics' favorites list
<p>The <code>num_games</code> column convinces us that our new list of the critics' top games reflects years that had quite a few well-reviewed games rather than just one or two hits. But which years dropped off the list due to having four or fewer reviewed games? Let's identify them so that someday we can track down more game reviews for those years and determine whether they might rightfully be considered as excellent years for video game releases!</p>


In [ ]:
%%sql 
SELECT year, round(avg(critic_score), 2) as avg_critic_score
    FROM game_sales as gs
inner join reviews as r
on r.game = gs.game
GROUP BY year
   
EXCEPT

SELECT year, round(avg(critic_score), 2) as avg_critic_score
    FROM game_sales as gs
inner join reviews as r
on r.game = gs.game
GROUP BY year
HAVING COUNT(DISTINCT r.game) > 4
ORDER BY avg_critic_score DESC
limit 6;

## 6. Years video game players loved
<p>It looks like the early 1990s might merit consideration as the golden age of video games based on <code>critic_score</code> alone, but we'd need to gather more games and reviews data to do further analysis. </p><p>Moving on to looking at the opinions of another important group of people: players!</p>

In [ ]:
%%sql 
SELECT year, round(avg(user_score), 2) as avg_user_score,COUNT(DISTINCT r.game) AS num_games
    FROM game_sales as gs
inner join reviews as r
on r.game = gs.game
    GROUP BY year
    HAVING COUNT(DISTINCT r.game) > 4
    ORDER BY avg_user_score DESC
    limit 10; 

## 7. Years that both players and critics loved
<p>We've got a list of the top ten years according to both critic reviews and user reviews. Are there any years that showed up on both tables? If so, those years would certainly be excellent ones!</p>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>
<p>We've also saved the results of our top user years query from the previous task into a table:</p>
<h3 id="top_user_years_more_than_four_games"><code>top_user_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_user_score</code></td>
<td>float</td>
<td>Average of all user scores for games released in that year</td>
</tr>
</tbody>
</table>

In [ ]:
%%sql 
select c.year 
    from top_critic_years_more_than_four_games as c
inner join top_user_years_more_than_four_games as u
on c.year = u.year
group by u.year, c.year
    order by c.avg_critic_score DESC, u.avg_user_score DESC
    limit 10;

## 8. Sales in the best video game years
<p>We've got three years that both users and critics agreed were in the top ten! What about video game makers? Were sales good? </p>

In [ ]:
%%sql 
SELECT year, sum(games_sold) AS total_games_sold
from game_sales
where year in
(
    SELECT year
    FROM top_user_years_more_than_four_games
    INTERSECT
    SELECT year
    FROM top_critic_years_more_than_four_games
) 
group by year
order by total_games_sold desc;